In [1]:
import pandas as pd
from jinja2 import Template, Environment, FileSystemLoader
import os
import subprocess
import bibtexparser

### Generate Markdown from Template

In [5]:
fn_pubs = '../markdown_generator/publications.tsv'
df_pubs = pd.read_csv(fn_pubs, sep='\t')
pubs = df_pubs.to_dict(orient='records')

In [41]:
# Load the Markdown template
with open('template.md') as file:
    template_content = file.read()

# Create a Jinja2 Template object
template = Template(template_content)

# Render the template with the publications data
markdown_output = template.render(publications=pubs)

# Save the rendered Markdown content to a file
with open('cv.md', 'w') as file:
    file.write(markdown_output)


### Try to use custom template for html

In [ ]:
# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Create a Jinja2 environment and load the template file
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('cv_template.html')

# Render the template with the Markdown content
rendered_content = template.render(content=markdown_content)

# Write the rendered content to a temporary Markdown file
with open('temp.md', 'w') as file:
    file.write(rendered_content)

# Convert the temporary Markdown file to HTML using Pandoc
subprocess.run(['pandoc', 'temp.md', '-o', 'cv.html'])

# Remove the temporary Markdown file
subprocess.run(['rm', 'temp.md'])


: 

### Change to use bibtex (not csv)

##### parse bibtex file

In [2]:
# Read the BibTeX file using bibtexparser
with open('papers.bib') as file:
    publications = bibtexparser.load(file).entries

with open('talks.bib') as file:
    talks = bibtexparser.load(file).entries


with open('invited.bib') as file:
    invited = bibtexparser.load(file).entries


with open('thesi.bib') as file:
    thesi = bibtexparser.load(file).entries


##### add publications / talks using jinja2

In [3]:
# Load the Markdown template
with open('template.md') as file:
    template_content = file.read()

# Create a Jinja2 Template object
template = Template(template_content)

# Render the template with the publications data
markdown_output = template.render(publications=publications, talks=talks)

# Save the rendered Markdown content to a file
with open('cv.md', 'w') as file:
    file.write(markdown_output)


##### bold every instance of me name

In [4]:
import re

def bold_string_in_markdown(markdown_content, target_string):
    markdown_new = markdown_content.replace(target_string, '**' + target_string + '**')

    return markdown_new

# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Bold the target string in the Markdown content
bolded_content = bold_string_in_markdown(markdown_content, 'Ragland, John')

# Save the bolded Markdown content to a new file
with open('cv.md', 'w') as file:
    file.write(bolded_content)


##### build html file from custom template cv_template.html

In [5]:
# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Create a Jinja2 environment and load the template file
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('cv_template.html')

# Render the template with the Markdown content
rendered_content = template.render(content=markdown_content)

# Write the rendered content to a temporary Markdown file
with open('temp.md', 'w') as file:
    file.write(rendered_content)

# Convert the temporary Markdown file to HTML using Pandoc
subprocess.run(['pandoc', 'temp.md', '-o', 'cv.html'])

# Remove the temporary Markdown file
subprocess.run(['rm', 'temp.md'])


CompletedProcess(args=['rm', 'temp.md'], returncode=0)

##### Build PDF from Markdown

In [6]:
# Read the Markdown content from file
with open('cv.md') as file:
    markdown_content = file.read()

# Create a Jinja2 environment and load the template file
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('cv_template.tex')

# Render the template with the Markdown content
rendered_content = template.render(content=markdown_content)

# Write the rendered content to a temporary Markdown file
with open('temp.md', 'w') as file:
    file.write(rendered_content)

# Convert the temporary Markdown file to HTML using Pandoc
subprocess.run(['pandoc', 'temp.md', '-o', 'cv.pdf'])

# Remove the temporary Markdown file
subprocess.run(['rm', 'temp.md'])


Error producing PDF.
! LaTeX Error: Can be used only in preamble.

See the LaTeX manual or LaTeX Companion for explanation.
Type  H <return>  for immediate help.
 ...                                              
                                                  
l.55 \documentclass



CompletedProcess(args=['rm', 'temp.md'], returncode=0)

## Add Zotero Integration

In [1]:
from pyzotero import zotero

In [2]:
# Set up your Zotero API credentials
library_id = '5442436'
library_type = 'user'  # 'user' or 'group'
api_key = '4beyxpmgVHhZavP45bkLf5vG'

In [3]:
# Create a Zotero API client
zot = zotero.Zotero(library_id, library_type, api_key)


In [18]:
# These keys can be found in the URL of the collection
talks_id = 'ZRKYJLDM'
papers_id = '8QEDMTRG'
thesi_id = '9PJCMLTU'
invited_id = 'I4375JL3'

# Get the items in the collection
talks = zot.collection_items(talks_id)
papers = zot.collection_items(papers_id)
thesi = zot.collection_items(thesi_id)
invited = zot.collection_items(invited_id)

#invited = zot.collection_items(invited_id, content='bib', style='bibtex')


In [26]:
for talk in talks:
    print(talk['data']['itemType'])

conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper
conferencePaper


In [14]:
papers = zot.collection_items(papers_id)
paper_attachments = []
for paper in papers:
    if paper['data']['itemType']:
        paper_attachments.append(paper)
        papers.remove(paper)


In [27]:
papers_bib_entries = []
for paper in papers:
    papers_bib_entries.append(zot.item(paper['data']['key'], content='bib', style='bibtex'))

['<div class="csl-entry"> @article{Ragland_Abadi_2022_Exploring surface source contributions to ocean ambient noise interferometry.pdf }</div>']

In [7]:
for k,i in enumerate(talks):

    start = i.index("@") 
    if k == 0:
        talks_bib = i[start:-6]
    else:
        talks_bib += i[start:-6]
    talks_bib += '\n'

for k, i in enumerate(papers):
    start = i.index("@")
    if k == 0:
        papers_bib = i[start:-6]
    else:
        papers_bib += i[start:-6]
    papers_bib += '\n'


for k, i in enumerate(thesi):
    start = i.index("@")
    if k == 0:
        thesi_bib = i[start:-6]
    else:
        thesi_bib += i[start:-6]
    thesi_bib += '\n'

for k, i in enumerate(invited):
    start = i.index("@")
    if k == 0:
        invited_bib = i[start:-6]
    else:
        invited_bib += i[start:-6]
    invited_bib += '\n'


In [10]:
talks_bib

'@inproceedings{Ragland_Schwock_Munson_Abadi_2021, address={Seattle, WA}, title={An overview of ambient sound using OOI hydrophone network}, volume={150}, url={https://doi.org/10.1121/10.0007594}, DOI={10.1121/10.0007594}, abstractNote={The NSF-funded Ocean Observatories Initiative (OOI) is a large-scale project that provides a unique observation of the ocean, collecting acoustic, meteorological, and oceanic datasets. Specifically, for hydrophone data, there are five low frequency hydrophones (Fs\u2009=\u2009200 Hz), and six broadband hydrophones (Fs\u2009=\u200964 kHz) dispersed around the north-east pacific. This acoustic dataset presents numerous potential areas of study in the field of ambient sound analysis. In this talk, we analysis 6 years of acoustic data to identify prominent features that are present in the OOI acoustic dataset. Some notable occurrences that are observed in the dataset include volcanic activity, rain and wind noise, fin whale vocalizations, and shipping event

In [11]:
# Save the BibTeX file
with open('talks.bib', 'w', encoding='utf-8') as file:
    file.write(talks_bib)
with open('papers.bib', 'w', encoding='utf-8') as file:
    file.write(papers_bib)
with open('thesi.bib', 'w', encoding='utf-8') as file:
    file.write(thesi_bib)
with open('invited.bib', 'w', encoding='utf-8') as file:
    file.write(invited_bib)


NameError: name 'invited_bib' is not defined

## Fix bugs with bibtex file

In [16]:
import bibtexparser
import re

In [18]:
def edit_citation_keys(bib_file_path):
    with open(bib_file_path, 'r') as bib_file:
        bib_data = bib_file.readlines()

    modified_bib_data = []
    for line in bib_data:
        if line.startswith('@'):
            # Extract the citation key from the line
            start_index = line.find('{') + 1
            end_index = line.find(',')
            citation_key = line[start_index:end_index].strip()

            # Remove spaces and periods from the citation key using regex
            citation_key = re.sub(r'[ .]', '', citation_key)

            # Modify the line with the updated citation key
            line = line[:start_index] + citation_key + line[end_index:]

        modified_bib_data.append(line)

    # Save the modified .bib file
    with open(bib_file_path, 'w') as bib_file:
        bib_file.writelines(modified_bib_data)

In [19]:

edit_citation_keys('papers.bib')


## Add attachment support

In [1]:
from pyzotero import zotero
import bibtexparser
from jinja2 import Template, Environment, FileSystemLoader
import re
import subprocess
#from ironpdf import *
import os
import cv_tools
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

In [2]:
# Define functions used in script
def edit_citation_keys(bib_file_path):
    with open(bib_file_path, 'r') as bib_file:
        bib_data = bib_file.readlines()

    modified_bib_data = []
    for line in bib_data:
        if line.startswith('@'):
            # Extract the citation key from the line
            start_index = line.find('{') + 1
            end_index = line.find(',')
            citation_key = line[start_index:end_index].strip()

            # Remove spaces and periods from the citation key using regex
            citation_key = re.sub(r'[ .]', '', citation_key)

            # Modify the line with the updated citation key
            line = line[:start_index] + citation_key + line[end_index:]

        modified_bib_data.append(line)

    # Save the modified .bib file
    with open(bib_file_path, 'w') as bib_file:
        bib_file.writelines(modified_bib_data)


print('loading citations from zotero...')

# Set up your Zotero API credentials (specific to user)
library_id = '5442436'
library_type = 'user'  # 'user' or 'group'
api_key = '4beyxpmgVHhZavP45bkLf5vG'

# Create a Zotero API client
zot = zotero.Zotero(library_id, library_type, api_key)

# These keys can be found in the URL of the collection
talks_id = 'ZRKYJLDM'
papers_id = '8QEDMTRG'
thesi_id = '9PJCMLTU'
invited_id = 'I4375JL3'

# Get the items in the collection
talks = zot.collection_items(talks_id)
papers = zot.collection_items(papers_id)
thesi = zot.collection_items(thesi_id)
invited = zot.collection_items(invited_id)

loading citations from zotero...


In [3]:

# Parse attachments and entries
talk_attachments = []
for talk in talks:
    if talk['data']['itemType'] == 'attachment':
        talk_attachments.append(talk)
        talks.remove(talk)

paper_attachments = []
for paper in papers:
    if paper['data']['itemType'] == 'attachment':
        paper_attachments.append(paper)
        papers.remove(paper)

thesi_attachments = []
for thesis in thesi:
    if thesis['data']['itemType'] == 'attachment':
        thesi_attachments.append(thesis)
        thesi.remove(thesis)

invited_attachments = []
for invite in invited:
    if invite['data']['itemType'] == 'attachment':
        invited_attachments.append(invite)
        invited.remove(invite)

# Create List of Bibliography Entries
talks_bib_entries = []
for talk in talks:
    talks_bib_entries.append(zot.item(talk['data']['key'], content='bib', style='bibtex'))

papers_bib_entries = []
for paper in papers:
    papers_bib_entries.append(zot.item(paper['data']['key'], content='bib', style='bibtex'))

thesi_bib_entries = []
for thesis in thesi:
    thesi_bib_entries.append(zot.item(thesis['data']['key'], content='bib', style='bibtex'))

invited_bib_entries = []
for invite in invited:
    invited_bib_entries.append(zot.item(invite['data']['key'], content='bib', style='bibtex'))

In [12]:
for paper in papers:
    print(paper['data']['title'])

Exploring surface source contributions to ocean ambient noise interferometry with airgun shots
An overview to modeling, characterizing, and predicting the effects of internal gravity waves on acoustic propagation at basin to global scales
Using ocean ambient sound to sense arrival time fluctuations due to temperature
Ragland et al_2022_An overview of ambient sound using Ocean Observatories Initiative hydrophones.pdf
An overview of ambient sound using Ocean Observatories Initiative hydrophones
Long-term noise interferometry analysis in the northeast Pacific Ocean


In [19]:
# add attachment files to bib entries
files_dir_url = 'https://john-ragland.github.io/files'
for k, paper in enumerate(papers):
    for paper_attachment in paper_attachments:
        if paper['key'] == paper_attachment['data']['parentItem']:
            fn = f"{files_dir_url}/{paper_attachment['data']['path'][12:].replace(' ','_')}"
            papers_bib_entries[k] = cv_tools.insert_string_before_last_brace(papers_bib_entries[k][0], f'filepath={ {fn} }')

In [36]:
zot_dir = '/Users/jhrag/Library/CloudStorage/OneDrive-UW/zotero_library/'
files_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'files/'))


In [34]:
def delete_files_in_directory(directory_path):
    if not os.path.exists(directory_path):
        print(f"Error: Directory '{directory_path}' does not exist.")
        return

    if not os.path.isdir(directory_path):
        print(f"Error: '{directory_path}' is not a valid directory path.")
        return

    file_list = os.listdir(directory_path)
    for file_name in file_list:
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):
            try:
                os.remove(file_path)
                print(f"Deleted file: {file_path}")
            except OSError as e:
                print(f"Error while deleting file: {file_path}\n{e}")

In [133]:
delete_files_in_directory(files_dir)

Deleted file: /Users/jhrag/Code/John-Ragland.github.io/files/Ragland_2020_Digital Simulation and Recreation of a Vacuum Tube Guitar Amp.pdf
Deleted file: /Users/jhrag/Code/John-Ragland.github.io/files/Ragland_Abadi_2023_Towards estimating water column properties using ambient noise interferometry.pdf
Deleted file: /Users/jhrag/Code/John-Ragland.github.io/files/Ragland_et_al_2022_Long-term_noise_interferometry_analysis_in_the_northeast_Pacific_Ocean.pdf
Deleted file: /Users/jhrag/Code/John-Ragland.github.io/files/Ragland_Abadi_2023_Towards_estimating_water_column_properties_using_ambient_noise_interferometry.pdf
Deleted file: /Users/jhrag/Code/John-Ragland.github.io/files/Ragland et al_2022_An overview of ambient sound using Ocean Observatories Initiative hydrophones2.pdf
Deleted file: /Users/jhrag/Code/John-Ragland.github.io/files/Ragland_Abadi_2022_Exploring surface source contributions to ocean ambient noise interferometry2.pdf
Deleted file: /Users/jhrag/Code/John-Ragland.github.io/f

In [134]:

for talk_attachment in talk_attachments:
    fn = talk_attachment['data']['path'][12:]
    os.system(
        f"cp '{zot_dir}{fn}' '{files_dir}/{fn.replace(' ','_')}'"
    )
for paper_attachment in paper_attachments:
    fn = paper_attachment['data']['path'][12:]
    os.system(
        f"cp '{zot_dir}{fn}' '{files_dir}/{fn.replace(' ','_')}'"
    )
for thesi_attachment in thesi_attachments:
    fn = thesi_attachment['data']['path'][12:]
    os.system(
        f"cp '{zot_dir}{fn}' '{files_dir}/{fn.replace(' ','_')}'"
    )
for invited_attachment in invited_attachments:
    fn = invited_attachment['data']['path'][12:]
    os.system(
        f"cp '{zot_dir}{fn}' '{files_dir}/{fn.replace(' ','_')}'"
    )